In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd

In [2]:
client_id = '514cc70f222d43fca53673e32b5e8e24'
client_secret = '2d82619557ce4d6db3e2caba753a927d'
redirect_uri = 'http://localhost:8080/callback'
scope = 'user-library-read user-read-recently-played'

In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirect_uri,
                                               scope=scope, open_browser=False))

In [10]:
results = sp.current_user_recently_played(limit=50)  # You can adjust the limit as needed

In [5]:
tracks = results['items']
track_ids = [track['track']['id'] for track in tracks]
played_at = [track['played_at'] for track in tracks]

# Fetch audio features for each track
features = sp.audio_features(track_ids)

In [6]:
df_tracks = pd.DataFrame(features)
df_tracks['played_at'] = pd.to_datetime(played_at)

In [7]:
df_tracks['sadness'] = 1 - df_tracks['valence']
df_tracks['date'] = df_tracks['played_at'].dt.date
daily_sadness = df_tracks.groupby('date').agg({'sadness': 'mean'}).reset_index()

In [8]:
# Select features and target
features = df_tracks[['danceability', 'energy', 'tempo', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence']]
target = df_tracks['sadness']

In [9]:
df_tracks.to_csv('music.csv', index=False)